## 해파리 주간동향 크롤링

### 1차 시도 - 2021년까지는 통함

※ - 어업인 해파리모니터링요원의 협조로 취합/분석한 자료임 -

※ 출현율은 이번주 어업인모니터링요원 응답자(258명) 중 해파리를 관찰한 사람 수를 백분율화한 값으로 대량출현의 판단 근거로 이용될 수 없음.

In [ ]:
# 날짜 리스트 만들기
from datetime import timedelta
from datetime import date

keyword_list = []


# 여기 날짜 수정 or input으로 받기

s_y, s_m, s_d = input('시작 날짜를 입력 (ex: 2021/12/01): ').split('/')
e_y, e_m, e_d = input('끝 날짜를 입력 (ex: 2022/06/30): ').split('/')


start = date(int(s_y), int(s_m), int(s_d))
end = date(int(e_y), int(e_m), int(e_d))


while end >= start:
    keyword_list.append(str(end).replace('-', '')+'00')
    end = end - timedelta(weeks=1)
    
    
    

    
# 그냥 정보 크롤링 부분
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


region = input('지역 입력 (ex: 부산) : ')


df = pd.DataFrame(columns = ['date', 'sea', 'htype', 'percent', 'freq'])
keyword_idx = []

for keyword in keyword_list:
    url = 'https://www.nifs.go.kr/jelly/jemoNewsView.jely?news_seq='+keyword
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    response = requests.get(url, headers = header)
    html_text = response.text
    
    if len(html_text) == 7043: # 없는 날짜면 길이가 7043 (틀이 정해져 있고, 그 틀의 길이)
        pass
        
    
    else:
        
        soup = BeautifulSoup(html_text, 'html.parser')

        table = soup.find_all('table', attrs={'class':'table_style_04'})
        tr = table[1].find_all('tr')
        print(tr)
        
        for idx in range(2, len(tr)):
            td = tr[idx].find_all('td')
            
            # 해파리 종류 - char
            htype = td[0].find('b').get_text()
            
            # 발견률 - floast
            percent = td[2].get_text().split('\r')[0]
            
            # 독성 -  char
            poison = td[3].get_text()
            
            # 해파리 종류 + 독성 - char
            h_type = htype + '_' + poison
            
            
            wheret = td[1].get_text().split('\r')
            text = ''
            
            for i in range(1, len(wheret)-1):
                text = text + wheret[i]
            
            text1 = text.split(' ○')
            
            for i in range(1, len(text1)):
                text2 = text1[i].split('   - ')
                
                for j in range(1, len(text2)):
                    if region in text2[j]: # 해당 지역 이름(ex.부산)이 해역 안에 있을 때
                        
                        seaname = text2[j]
                        
                        if '고밀도' in text2[0]:
                            freq = 2 # 고밀도 출현 해역이면 2 - int
                        else:
                            freq = 1 # 저밀도 출현 해역이면 1 - int
                            
                            
                        # 데이터프레임에 넣어주는 작업
                        to_insert = {'date' : keyword[:-2],
                                     'sea' : seaname,
                                     'htype' : h_type,
                                     'percent' : percent,
                                     'freq' : freq}
                        
                        df = df.append(to_insert, ignore_index = True)
                        
                        
    
                        
df       




In [ ]:
td

In [ ]:
seaname

In [ ]:
df

### 2차 시도 - 2017년까지 괜찮은 듯. 해역 이름 파악 필요

In [ ]:
# 날짜 리스트 만들기
from datetime import timedelta
from datetime import date

keyword_list = []


# 여기 날짜 수정 or input으로 받기

s_y, s_m, s_d = input('시작 날짜를 입력 (ex: 2021/12/01): ').split('/')
e_y, e_m, e_d = input('끝 날짜를 입력 (ex: 2022/06/30): ').split('/')


start = date(int(s_y), int(s_m), int(s_d))
end = date(int(e_y), int(e_m), int(e_d))


while end >= start:
    keyword_list.append(str(end).replace('-', '')+'00')
    end = end - timedelta(weeks=1)
    
    

    
# 그냥 정보 크롤링 부분
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re


region = input('지역 입력 (ex: 부산) : ')


df = pd.DataFrame(columns = ['date', 'sea', 'htype', 'percent', 'freq'])
keyword_idx = []

for keyword in keyword_list:
    url = 'https://www.nifs.go.kr/jelly/jemoNewsView.jely?news_seq='+keyword
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    response = requests.get(url, headers = header)
    html_text = response.text
    
    if len(html_text) == 7043: # 없는 날짜면 길이가 7043 (틀이 정해져 있고, 그 틀의 길이)
        pass
        
    
    else:
        
        soup = BeautifulSoup(html_text, 'html.parser')

        table = soup.find_all('table', attrs={'class':'table_style_04'})
        tr = table[1].find_all('tr')
        
        for idx in range(2, len(tr)):
            td = tr[idx].find_all('td')
            
            # 해파리 종류 - char
            htype = td[0].find('b').get_text()
            
            # 발견률 - floast
            percent = float(re.findall('\d{0,2}[.]{0,1}\d{0,2}[%]', td[2].get_text().split('\r')[0])[0].replace('%', ''))
            
            # 독성 -  char
            poison = td[3].get_text()
            
            # 해파리 종류 + 독성 - char
            h_type = htype + '_' + poison
            
            
            wheret = td[1].get_text().split('\r')
            
            for i in range(len(wheret)):
                if region in wheret[i] and '출현' not in wheret[i]:

                    
                    seaname = wheret[i]
                    freq = 0 # 고밀도 저밀도 없으면 0
                    
                    for j in range(i-1, -1, -1):
                        
                        if '고밀도' in wheret[j]:
                            freq = 2
                            break
                            
                        elif '저밀도' in wheret[j]:
                            freq = 1
                            break
                            
                        
                            
                    # 데이터프레임에 넣어주는 작업
                    to_insert = {'date' : keyword[:-2],
                                 'sea' : seaname,
                                 'htype' : h_type,
                                 'percent' : percent,
                                 'freq' : freq}
                        
                        
                    df = df.append(to_insert, ignore_index = True)
                    
                    
        print(keyword + ' is done.')
                            
                        
                        
                        
df            

In [ ]:
df

### 해역 이름 수정

In [120]:
# 날짜 리스트 만들기
from datetime import timedelta
from datetime import date

keyword_list = []


# 여기 날짜 수정 or input으로 받기

s_y, s_m, s_d = input('시작 날짜를 입력 (ex: 2021/12/01): ').split('/')
e_y, e_m, e_d = input('끝 날짜를 입력 - 존재하는 페이지 날짜로 (ex: 2022/06/30): ').split('/')


start = date(int(s_y), int(s_m), int(s_d))
end = date(int(e_y), int(e_m), int(e_d))


while end >= start:
    keyword_list.append(str(end).replace('-', '')+'00')
    end = end - timedelta(weeks=1)
    
    

    
# 그냥 정보 크롤링 부분
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re


# 2차원 리스트 -> 1차원 리스트
def flatten(lst):
    result = []
    for item in lst:
        result.extend(item)
    return result



region = input('지역 입력 (ex: 부산) : ')


df = pd.DataFrame(columns = ['date', 'sea', 'htype', 'percent', 'freq'])
keyword_idx = []

for keyword in keyword_list:
    url = 'https://www.nifs.go.kr/jelly/jemoNewsView.jely?news_seq='+keyword
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    response = requests.get(url, headers = header)
    html_text = response.text
    
    if len(html_text) == 7043: # 없는 날짜면 길이가 7043 (틀이 정해져 있고, 그 틀의 길이)
        pass
        
    
    else:
        
        soup = BeautifulSoup(html_text, 'html.parser')

        table = soup.find_all('table', attrs={'class':'table_style_04'})
        tr = table[1].find_all('tr')
        
        for idx in range(2, len(tr)):
            td = tr[idx].find_all('td')
            
            # 해파리 종류 - char
            htype = td[0].find('b').get_text()
            
            # 발견률 - floast
            percent = float(re.findall('\d{0,2}[.]{0,1}\d{0,2}[%]', td[2].get_text().split('\r')[0])[0].replace('%', ''))
            
            # 독성 -  char
            poison = td[3].get_text()
            
            # 해파리 종류 + 독성 - char
            h_type = htype + '_' + poison
            
            
            wheret = td[1].get_text().split('\r')
            
            
            ## 예외처리 2018060700
            if '？' in wheret[0]:
                for i in range(len(wheret)):
                    wheret[i] = wheret[i].replace('？', ' ')
                    wheret[i] = wheret[i].replace('\n', '')

                    if '고밀도' in wheret[i] or '저밀도' in wheret[i]:
                        wheret[i] = '○' + wheret[i]

                    if '○' not in wheret[i] and '-' not in wheret[i]:
                        wheret[i] = '-' + wheret[i]

                wheret = [i for i in wheret if i != '-']

            
            
            
            for i in range(len(wheret)):
                if '(' in wheret[i]:
                    wheret[i] = re.sub(r'\([^)]*\)', '', wheret[i])
            
            
            ## 해역 이름 수정
            
            wheret = [i.strip() for i  in wheret]
            wheret = [i.replace('.', ',') for i  in wheret]
            wheret = [i for i in wheret if i !='']
            wheret = [i for i in wheret if '※' not in i]
            
            # 어떤 날짜에서는 , 다음에 -붙은채로 이어져야 하는게 있는 경우도... 수정
            for i in range(len(wheret)):
                
                if wheret[i][-1] == ',' and '○' not in wheret[i] and region in wheret[i+1]:
                    wheret[i] = wheret[i].replace(',', '')
                elif wheret[i][-1] == ',' and '○' not in wheret[i]:
                    wheret[i+1] = wheret[i+1].replace('-', '')
                else:
                    pass
                    
            
            # - 없는 애들 다 붙이기
            for t in range(len(wheret)-1, -1, -1):

                if '-' in wheret[t] or '•' in wheret[t] or '○' in wheret[t] or '◎' in wheret[t]:
                    pass
                else:
                    wheret[t-1] = wheret[t-1] +', '+ wheret[t]
                    wheret[t] = 0
                    
            wheret1 = [i for i in wheret if i != 0]
            
            
            wheret1 = [i.replace('․', '') for i in wheret1]
            wheret1 = [i.replace(',,', ',') for i in wheret1]
            
            
            
            # 합친 애들 다시 나눠서 이름 예쁘게 붙여주기
            index = []
            for i in range(len(wheret1)):
                if '-' in wheret1[i]:
                    index.append(i)


            wheret2 = [i.split(', ') for i in wheret1]
            
            for i in index:
    
                if len(wheret2[i][0].split(' ')) >= 4:
                    middle = wheret2[i][0].split(' ')[2] + ' '
                else:
                    middle = ''
                    
                
                for j in range(1, len(wheret2[i])):
                

                    if len(wheret2[i][j].split(' ')) >= 2:
                        if region in wheret2[i][j].split(' '):
                            pass
                        else:
                            wheret2[i][j] = '- ' + wheret2[i][0].split(' ')[1] +' '+ wheret2[i][j]
                            middle = wheret2[i][j].split(' ')[2] +' '

                    if len(wheret2[i][j].split(' ')) == 1:
                        wheret2[i][j] = '- ' + wheret2[i][0].split(' ')[1] +' '+ middle+ wheret2[i][j]

            
            wheret4 = flatten(wheret2)
            
            
            # 중복되는 애들 제거
            wheret5 = wheret4
        
            for i in range(len(wheret5)):
                if '-' in wheret5[i]:
                    for j in range(i+1, len(wheret5)):
                        if '-' in wheret5[j]:
                            if wheret5[i] == wheret5[j]:
                                wheret5[j] = '0'
                        else:
                            break

            
            wheret6 = [item for item in wheret5 if item != '0']
            
            # 이름 정리 완성본
            f_wheret = wheret6
            f_wheret = [item.replace('  ', ' ') for item in f_wheret]
            
            
            # stopword 선정 후 제거
            f_wheret = [i.replace(' 연안', '') for i in f_wheret]
            f_wheret = [i.replace(' 부근', '') for i in f_wheret]
            f_wheret = [i.replace(' 등대', '') for i in f_wheret]
            f_wheret = [i.replace(' 연근해', '') for i in f_wheret]
            f_wheret = [i.replace(' 근해', '') for i in f_wheret]
            f_wheret = [i.replace('앞', '') for i in f_wheret]
            f_wheret = [i.replace('해변', '') for i in f_wheret]
            f_wheret = [i.replace('해수욕장', '') for i in f_wheret]
            
            
            f_wheret = [item.replace('-', '').strip() for item in f_wheret]
            
            
            
            
            
            
            for i in range(len(f_wheret)):
                if region in f_wheret[i] and '출현' not in f_wheret[i]:

                    
                    seaname = f_wheret[i]
                    freq = 0 # 고밀도 저밀도 없으면 0
                    
                    for j in range(i-1, -1, -1):
                        
                        if '고밀도' in f_wheret[j]:
                            freq = 2
                            break
                            
                        elif '저밀도' in f_wheret[j]:
                            freq = 1
                            break
                            
                        
                            
                    # 데이터프레임에 넣어주는 작업
                    to_insert = {'date' : keyword[:-2],
                                 'sea' : seaname,
                                 'htype' : h_type,
                                 'percent' : percent,
                                 'freq' : freq}
                        
                        
                    df = df.append(to_insert, ignore_index = True)
                    
                    
        print(keyword + ' is done.')
                            
                        
                        
                        
df            

시작 날짜를 입력 (ex: 2021/12/01): 2017/03/01
끝 날짜를 입력 - 존재하는 페이지 날짜로 (ex: 2022/06/30): 2022/06/30
지역 입력 (ex: 부산) : 부산
2022063000 is done.
2022062300 is done.
2022061600 is done.
2022060900 is done.
2022060200 is done.
2022052600 is done.
2022051900 is done.
2022051200 is done.
2021120900 is done.
2021120200 is done.
2021112500 is done.
2021111800 is done.
2021111100 is done.
2021110400 is done.
2021102800 is done.
2021102100 is done.
2021101400 is done.
2021100700 is done.
2021093000 is done.
2021092300 is done.
2021091600 is done.
2021090900 is done.
2021090200 is done.
2021082600 is done.
2021081900 is done.
2021081200 is done.
2021080500 is done.
2021072900 is done.
2021072200 is done.
2021071500 is done.
2021070800 is done.
2021070100 is done.
2021062400 is done.
2021061700 is done.
2021061000 is done.
2021060300 is done.
2021052700 is done.
2021052000 is done.
2021051300 is done.
2021050600 is done.
2020121000 is done.
2020120300 is done.
2020112600 is done.
2020111900 is done.
20201112

,date,sea,htype,percent,freq
0,20220630,부산 태종대,노무라입깃해파리_강독성,8.00,1
1,20220630,부산 북형제도,노무라입깃해파리_강독성,8.00,1
2,20220630,부산 가덕도,보름달물해파리_약독성,35.70,1
3,20220630,부산 영도,보름달물해파리_약독성,35.70,1
4,20220630,부산 광안리,보름달물해파리_약독성,35.70,1
...,...,...,...,...,...
851,20170525,부산 생도,보름달물해파리_약독성,11.54,1
852,20170525,부산 광안리,보름달물해파리_약독성,11.54,1
853,20170518,부산 생도,보름달물해파리_약독성,11.54,2
854,20170511,부산 고리원전,보름달물해파리_약독성,11.54,1


In [121]:
df.sea.unique()

array(['부산 태종대', '부산 북형제도', '부산 가덕도', '부산 영도', '부산 광안리', '부산 광안대교',
       '부산 기장 대변항', '부산 오륙도', '부산 해운대 청사포', '부산 일광면 문동', '부산 기장 칠암리',
       '부산 강서 가덕도', '부산 영도 태종대', '부산 기장 일광면', '부산 기장 월내항', '부산 청사포',
       '부산 대변항', '부산 암남동', '부산 송정', '부산 강개', '부산 해운대', '부산 수영만', '부산 월내항',
       '부산 오륙도송정 사이', '부산 공수방파제', '부산 눌차도', '부산 동백섬', '부산 일광면',
       '부산 민락오륙도 사이', '부산 남형제도', '부산 태종도', '부산 목도', '부산 북형제도 1마일',
       '부산 다대포항', '부산 해운대청사포 사이', '부산 다대항', '부산 오륙도송정 청사포 사이', '부산 광안',
       '부산 송도', '부산 신항만', '부산 다대동', '부산 태종도영도 사이', '부산 다대포', '부산 기장공수마을',
       '부산 생도', '부산 기장 고리마을', '부산 기장 공수마을', '부산 영도 생도', '부산 해운대 동백섬',
       '부산 해운대 태종대', '부산 송도·송정·해운대', '부산 송도·광안리·송정', '부산 기장 청사포',
       '부산 동삼동', '부산 오륙도송정', '부산 가덕도 눌차만', '부산 기장 원전고리', '부산 영도 생도부근',
       '부산 오륙도송정청사포', '부산 고리원전', '부산 송도 생도', '부산 동백리', '부산 기장 동수',
       '부산 오륙도 청사포', '부산 기장 고리원전', '부산 영도 오륙도', '부산 시랑리', '부산 학리',
       '부산 영도등대', '부산 목도 대항동', '부산 오륙도 다대동', '부산 오륙도 공수', '부산 고리원자력 발전소',
       '부산 대항동', '부산 생도 오륙

In [122]:
df.freq.unique()

array([1, 2, 0], dtype=object)

In [123]:
df[df.date == '20180614']

,date,sea,htype,percent,freq
697,20180614,부산 해운대,보름달물해파리_약독성,19.05,2
698,20180614,부산 광안리,보름달물해파리_약독성,19.05,1
699,20180614,부산 생도,보름달물해파리_약독성,19.05,1
700,20180614,부산 목도,보름달물해파리_약독성,19.05,1
701,20180614,부산 목도,유령해파리_강독성,0.44,1


In [127]:
## csv 파일로 저장

df.to_csv('해파리주간동향_' + region + '_' + s_y+s_m+s_d + '_' + e_y+e_m+e_d + '.csv', 
          encoding = 'utf-8', index=False)
print('해파리주간동향_' + region + '_' + s_y+s_m+s_d + '_' + e_y+e_m+e_d + '.csv 저장 완료')

해파리주간동향_부산_20170301_20220630.csv 저장 완료


In [128]:
## 잘 불러지는 지 확인

data = pd.read_csv('해파리주간동향_부산_20170301_20220630.csv', encoding = 'utf-8')
data

,date,sea,htype,percent,freq
0,20220630,부산 태종대,노무라입깃해파리_강독성,8.00,1
1,20220630,부산 북형제도,노무라입깃해파리_강독성,8.00,1
2,20220630,부산 가덕도,보름달물해파리_약독성,35.70,1
3,20220630,부산 영도,보름달물해파리_약독성,35.70,1
4,20220630,부산 광안리,보름달물해파리_약독성,35.70,1
...,...,...,...,...,...
851,20170525,부산 생도,보름달물해파리_약독성,11.54,1
852,20170525,부산 광안리,보름달물해파리_약독성,11.54,1
853,20170518,부산 생도,보름달물해파리_약독성,11.54,2
854,20170511,부산 고리원전,보름달물해파리_약독성,11.54,1


#### 부산부산 & 기타 등등 이상한 애들 확인

In [59]:
a = td[1].get_text().split('\r')
for i in range(len(a)):
    a[i] = a[i].replace('？', ' ')
    a[i] = a[i].replace('\n', '')
    
    if '고밀도' in a[i] or '저밀도' in a[i]:
        a[i] = '○' + a[i]
        
    if '○' not in a[i] and '-' not in a[i]:
        a[i] = '-' + a[i]

a = [i for i in a if i != '-']
        
a

['○동해 출현',
 '○  고밀도 출현 해역',
 '   - 충남 가로림만 부근',
 '   - 경북 포항 구롱포항 연안',
 '-     강원 묵호항, 동산항, 남애항 연안',
 '○  저밀도 출현 해역',
 '   - 경북 울산 정자 앞바다',
 '-     경북 포항 영일만 연안',
 '-     경북 감포, 덕천리 연안',
 '   -   강원 동해 용정동 연안',
 '-      강원 양양 수산항 연안']

In [36]:
for i in range(len(wheret)):
    if wheret[i][-1] == ',':
        print(i, wheret[i])
        wheret[i+1] = wheret[i+1].replace('-', '')

wheret

9 - 경남 통영 두미도, 사량도,
13 - 울산 주전동, 방어동, 정자항,
17 - 경북 포항 구만리, 강사리,
21 - 경북 울진 동정항, 사동항,
40 - 부산 광안리해수욕장,


['○ 서해․남해․동해․제주 출현',
 '•고밀도 출현 해역',
 '- 전남 신안 흑산도-진도 사이',
 '- 전남 진도 조도면 독거도 연안',
 '- 전남 고흥 나로도 근해',
 '- 경남 남해 미조면 연근해',
 '- 경남 하동 대도 연안',
 '- 경남 거제 학동, 외도 연안',
 '- 경남 거제 북여도 근해',
 '- 경남 통영 두미도, 사량도,',
 ' 수우도, 욕지도 연안',
 '- 부산 영도 연근해',
 '- 부산 가덕도, 오륙도, 다대항 연안',
 '- 울산 주전동, 방어동, 정자항,',
 ' 울기등대 연안',
 '- 경북 경주 월성원자력발전소 부근',
 '- 경북 포항 호미곶 연근해',
 '- 경북 포항 구만리, 강사리,',
 '구룡포항, 이가리항 연안',
 '- 경북 영덕 축산항, 대진항 연안',
 '- 경북 울진 후포리 연근해',
 '- 경북 울진 동정항, 사동항,',
 ' 후정해변 연안',
 '- 경북 울릉도-강원 묵호항 사이',
 '- 강원 삼척 부남리 연안',
 '- 강원 삼척 대진항 근해',
 '- 강원 동해 묵호항 연근해',
 '- 강원 동해 대진항 연안',
 '- 강원 강릉 주문진 연안',
 '- 강원 양양 남애항 연안',
 '- 강원 고성 명호리 연안',
 '•저밀도 출현 해역',
 '- 충남 서산 고파도 연안',
 '- 전남 영광 칠산도 연안',
 '- 전남 신안 우이도 연안',
 '- 전남 해남 송지면 땅끝마을 연안',
 '- 전남 완도 약산면-신지면 사이',
 '- 전남 여수 남면 부근',
 '- 경남 사천 사천만 부근',
 '- 경남 창원 잠도 연안',
 '- 부산 광안리해수욕장,',
 ' 청사포, 해운대, 신항만 부근',
 '- 부산 가덕도 근해',
 '- 울산 동구 근해',
 '- 경북 포항 대동배1리 지선',
 '- 경북 포항 칠포항 근해',
 '- 제주 차귀도 연안',
 '- 제주 제주시 근해, ']

In [29]:
wheret2

[['○ 서해․남해․동해․제주 출현'],
 ['•고밀도 출현 해역'],
 ['- 전남 신안 흑산도-진도 사이'],
 ['- 전남 진도 조도면 독거도'],
 ['- 전남 고흥 나로도 근해'],
 ['- 경남 남해 미조면 연근해'],
 ['- 경남 하동 대도'],
 ['- 경남 거제 학동', '- 경남 - 경남 - 경남 거제 외도'],
 ['- 경남 거제 북여도 근해'],
 ['- 경남 통영 두미도', '- 경남 - 경남 - 경남 통영 사량도,'],
 ['- 수우도', '- 수우도 - 수우도 - 수우도 욕지도'],
 ['- 부산 영도 연근해'],
 ['- 부산 가덕도', '- 부산 - 부산 - 부산 오륙도', '- 부산 - 부산 - 부산 다대항'],
 ['- 울산 주전동', '- 울산 - 울산 - 울산 방어동', '- 울산 - 울산 - 울산 정자항,'],
 ['- 울기등대'],
 ['- 경북 경주 월성원자력발전소'],
 ['- 경북 포항 호미곶 연근해'],
 ['- 경북 포항 구만리', '- 경북 - 경북 - 경북 포항 강사리,'],
 ['-구룡포항', '이가리항'],
 ['- 경북 영덕 축산항', '대진항'],
 ['- 경북 울진 후포리 연근해'],
 ['- 경북 울진 동정항', '사동항,'],
 ['- 후정해변'],
 ['- 경북 울릉도-강원 묵호항 사이'],
 ['- 강원 삼척 부남리'],
 ['- 강원 삼척 대진항 근해'],
 ['- 강원 동해 묵호항 연근해'],
 ['- 강원 동해 대진항'],
 ['- 강원 강릉 주문진'],
 ['- 강원 양양 남애항'],
 ['- 강원 고성 명호리'],
 ['•저밀도 출현 해역'],
 ['- 충남 서산 고파도'],
 ['- 전남 영광 칠산도'],
 ['- 전남 신안 우이도'],
 ['- 전남 해남 송지면 땅끝마을'],
 ['- 전남 완도 약산면-신지면 사이'],
 ['- 전남 여수 남면'],
 ['- 경남 사천 사천만'],
 ['- 경남 창원 잠도'],
 ['- 부산 광안리해수욕장,'],
 ['- 청사포', '해운대',

In [28]:
for i in index:
    if len(wheret2[i][0].split(' ')) >= 4:
        middle = wheret2[i][0].split(' ')[2] + ' '
    else:
        middle = ''


    for j in range(1, len(wheret2[i])):
        if len(wheret2[i][j].split(' ')) >= 2:
            wheret2[i][j] = '- ' + wheret2[i][0].split(' ')[1] +' '+ wheret2[i][j]
            middle = wheret2[i][j].split(' ')[2] +' '

        if len(wheret2[i][j].split(' ')) == 1:
            print(i, j, wheret2[i][j])
            wheret2[i][j] = '- ' + wheret2[i][0].split(' ')[1] +' '+ middle+ wheret2[i][j]

            
wheret4 = flatten(wheret2)
wheret4

18 1 이가리항


IndexError: list index out of range

### 확인용 코드들

In [3]:
# 날짜 리스트 만들기
from datetime import timedelta
from datetime import date

keyword_list = []


# 여기 날짜 수정 or input으로 받기

s_y, s_m, s_d = input('시작 날짜를 입력 (ex: 2021/12/01): ').split('/')
e_y, e_m, e_d = input('끝 날짜를 입력 (ex: 2022/06/30): ').split('/')


start = date(int(s_y), int(s_m), int(s_d))
end = date(int(e_y), int(e_m), int(e_d))


while end >= start:
    keyword_list.append(str(end).replace('-', '')+'00')
    end = end - timedelta(weeks=1)


시작 날짜를 입력 (ex: 2021/12/01): 2017/07/19
끝 날짜를 입력 (ex: 2022/06/30): 2017/07/20


In [ ]:
keyword_list

In [5]:
# 그냥 정보 크롤링 부분
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


region = input('지역 입력 (ex: 부산) : ')


df = pd.DataFrame(columns = ['date', 'sea', 'htype', 'percent', 'freq'])
keyword_idx = []

for keyword in keyword_list:
    url = 'https://www.nifs.go.kr/jelly/jemoNewsView.jely?news_seq='+keyword
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    response = requests.get(url, headers = header)
    html_text = response.text
    
    if len(html_text) == 7043: # 없는 날짜면 길이가 7043 (틀이 정해져 있고, 그 틀의 길이)
        pass
        
    
    else:
        
        soup = BeautifulSoup(html_text, 'html.parser')

        table = soup.find_all('table', attrs={'class':'table_style_04'})
        tr = table[1].find_all('tr')
        
        for idx in range(2, len(tr)):
            td = tr[idx].find_all('td')
            
            # 해파리 종류 - char
            htype = td[0].find('b').get_text()
            
            # 발견률 - floast
            percent = float(re.findall('\d{0,2}[.]{0,1}\d{0,2}[%]', td[2].get_text().split('\r')[0])[0].replace('%', ''))
            
            # 독성 -  char
            poison = td[3].get_text()
            
            # 해파리 종류 + 독성 - char
            h_type = htype + '_' + poison
            
            
            wheret = td[1].get_text().split('\r')
            
            for i in range(len(wheret)):
                wheret[i] = wheret[i].strip()
                
            #for i in range(len(wheret)):
                
            
                
            for i in range(len(wheret)):     
                
                if region in wheret[i] and '출현' not in wheret[i]:

                    
                    seaname = wheret[i]
                    freq = 0
                    
                    for j in range(i-1, -1, -1):
                        
                        if '고밀도' in wheret[j]:
                            freq = 2
                            break
                            
                        elif '저밀도' in wheret[j]:
                            freq = 1
                            break
                            
                        
                            
                    # 데이터프레임에 넣어주는 작업
                    to_insert = {'date' : keyword[:-2],
                                 'sea' : seaname,
                                 'htype' : h_type,
                                 'percent' : percent,
                                 'freq' : freq}
                        
                        
                    df = df.append(to_insert, ignore_index = True)
        print(keyword + ' is done.')
                            
                        
                        
                        
df        

지역 입력 (ex: 부산) : 부산


NameError: name 're' is not defined

In [ ]:
tr[3].find_all('td')[1].get_text().split('\r')[5].strip()

In [ ]:
df.sea.unique()

In [ ]:
df.htype.unique()

In [ ]:
df.freq.unique()

In [ ]:
a = '7/27(0%)'
b ='7/27(3.18%)'

re.findall('\d{0,2}[.]{0,1}\d{0,2}[%]', a)
#re.findall('\d{1,2}[.]\d\d[%]', b)

In [ ]:
td[2].get_text().split('\r')[0]

In [ ]:
for i in range(3, -1, -1):
    print(i)

#### 바다 해역 이름 수정하기 test

In [6]:
wheret = table[1].find_all('tr')[3].find_all('td')[1].get_text().split('\r')

text = ''
            
for i in range(1, len(wheret)-1):
    text = text + wheret[i]
    
text.split('출현')


[' •고밀도 ',
 ' 해역   - 충남 당진 난지도 부근,      태안 황도리, 서천 마량리 연안   - 전북 군산 십이동파도,      새만금 내측·외측, 비응도 연안,      신시도, 무녀도 부근       부안 치도리, 진리 연안   - 전남 고흥 도천리, 안남리 연안     거금도 부근     보성 비봉리, 회천면 연안     영암 삼호읍 연안   - 부산 기장 동수 연안   - 경남 고성 수월리, 덕명리, 당항포 연안     창원 잠도, 수도 부근, 행암동 난포리 연안      거제 지심도 부근, 구조라해변,      사등면 연안   - 경북 울진 오산항 연안  •저밀도 ',
 ' 해역   - 충남 보령 외연도, 태안 옹도 부근     서산 부석면 연안   - 전북 군산 어청도 부근   - 전남 진도 원포리, 완도 월양리 연안,   - 경남 고성 송천리, 남해 진목리 연안     통영 욕지도 부근 수월리 연안     창원 심리, 요장리, 덕동동 연안   - 부산 가덕도 부근   - 울산 주전동 연안   - 경북 포항 호미곶면, 남호리, 지경리 연안     울진 덕천리 연안, 경주 감포 연안']

In [7]:
wheret

['○동․서․남동해 출현 ',
 ' •고밀도 출현 해역',
 '   - 충남 당진 난지도 부근, ',
 '     태안 황도리, 서천 마량리 연안',
 '   - 전북 군산 십이동파도, ',
 '     새만금 내측·외측, 비응도 연안, ',
 '     신시도, 무녀도 부근  ',
 '     부안 치도리, 진리 연안',
 '   - 전남 고흥 도천리, 안남리 연안',
 '     거금도 부근',
 '     보성 비봉리, 회천면 연안',
 '     영암 삼호읍 연안',
 '   - 부산 기장 동수 연안',
 '   - 경남 고성 수월리, 덕명리, 당항포 연안',
 '     창원 잠도, 수도 부근, 행암동 난포리 연안',
 '      거제 지심도 부근, 구조라해변, ',
 '     사등면 연안',
 '   - 경북 울진 오산항 연안',
 '  •저밀도 출현 해역',
 '   - 충남 보령 외연도, 태안 옹도 부근',
 '     서산 부석면 연안',
 '   - 전북 군산 어청도 부근',
 '   - 전남 진도 원포리, 완도 월양리 연안,',
 '   - 경남 고성 송천리, 남해 진목리 연안',
 '     통영 욕지도 부근 수월리 연안',
 '     창원 심리, 요장리, 덕동동 연안',
 '   - 부산 가덕도 부근',
 '   - 울산 주전동 연안',
 '   - 경북 포항 호미곶면, 남호리, 지경리 연안',
 '     울진 덕천리 연안, 경주 감포 연안',
 '   - 강원 속초 용촌리 연안, 강릉 주문진 연안']

In [8]:
wheret = [i.strip() for i  in wheret]
wheret

['○동․서․남동해 출현',
 '•고밀도 출현 해역',
 '- 충남 당진 난지도 부근,',
 '태안 황도리, 서천 마량리 연안',
 '- 전북 군산 십이동파도,',
 '새만금 내측·외측, 비응도 연안,',
 '신시도, 무녀도 부근',
 '부안 치도리, 진리 연안',
 '- 전남 고흥 도천리, 안남리 연안',
 '거금도 부근',
 '보성 비봉리, 회천면 연안',
 '영암 삼호읍 연안',
 '- 부산 기장 동수 연안',
 '- 경남 고성 수월리, 덕명리, 당항포 연안',
 '창원 잠도, 수도 부근, 행암동 난포리 연안',
 '거제 지심도 부근, 구조라해변,',
 '사등면 연안',
 '- 경북 울진 오산항 연안',
 '•저밀도 출현 해역',
 '- 충남 보령 외연도, 태안 옹도 부근',
 '서산 부석면 연안',
 '- 전북 군산 어청도 부근',
 '- 전남 진도 원포리, 완도 월양리 연안,',
 '- 경남 고성 송천리, 남해 진목리 연안',
 '통영 욕지도 부근 수월리 연안',
 '창원 심리, 요장리, 덕동동 연안',
 '- 부산 가덕도 부근',
 '- 울산 주전동 연안',
 '- 경북 포항 호미곶면, 남호리, 지경리 연안',
 '울진 덕천리 연안, 경주 감포 연안',
 '- 강원 속초 용촌리 연안, 강릉 주문진 연안']

In [9]:
idx = []
for t in range(len(wheret)-1, -1, -1):

    if '-' in wheret[t] or '•' in wheret[t] or '○' in wheret[t] or '◎' in wheret[t]:
        pass
    else:
        wheret[t-1] = wheret[t-1] +', '+ wheret[t]
        wheret[t] = 0

wheret

['○동․서․남동해 출현',
 '•고밀도 출현 해역',
 '- 충남 당진 난지도 부근,, 태안 황도리, 서천 마량리 연안',
 0,
 '- 전북 군산 십이동파도,, 새만금 내측·외측, 비응도 연안,, 신시도, 무녀도 부근, 부안 치도리, 진리 연안',
 0,
 0,
 0,
 '- 전남 고흥 도천리, 안남리 연안, 거금도 부근, 보성 비봉리, 회천면 연안, 영암 삼호읍 연안',
 0,
 0,
 0,
 '- 부산 기장 동수 연안',
 '- 경남 고성 수월리, 덕명리, 당항포 연안, 창원 잠도, 수도 부근, 행암동 난포리 연안, 거제 지심도 부근, 구조라해변,, 사등면 연안',
 0,
 0,
 0,
 '- 경북 울진 오산항 연안',
 '•저밀도 출현 해역',
 '- 충남 보령 외연도, 태안 옹도 부근, 서산 부석면 연안',
 0,
 '- 전북 군산 어청도 부근',
 '- 전남 진도 원포리, 완도 월양리 연안,',
 '- 경남 고성 송천리, 남해 진목리 연안, 통영 욕지도 부근 수월리 연안, 창원 심리, 요장리, 덕동동 연안',
 0,
 0,
 '- 부산 가덕도 부근',
 '- 울산 주전동 연안',
 '- 경북 포항 호미곶면, 남호리, 지경리 연안, 울진 덕천리 연안, 경주 감포 연안',
 0,
 '- 강원 속초 용촌리 연안, 강릉 주문진 연안']

In [10]:
wheret = [i for i in wheret if i != 0]
wheret

['○동․서․남동해 출현',
 '•고밀도 출현 해역',
 '- 충남 당진 난지도 부근,, 태안 황도리, 서천 마량리 연안',
 '- 전북 군산 십이동파도,, 새만금 내측·외측, 비응도 연안,, 신시도, 무녀도 부근, 부안 치도리, 진리 연안',
 '- 전남 고흥 도천리, 안남리 연안, 거금도 부근, 보성 비봉리, 회천면 연안, 영암 삼호읍 연안',
 '- 부산 기장 동수 연안',
 '- 경남 고성 수월리, 덕명리, 당항포 연안, 창원 잠도, 수도 부근, 행암동 난포리 연안, 거제 지심도 부근, 구조라해변,, 사등면 연안',
 '- 경북 울진 오산항 연안',
 '•저밀도 출현 해역',
 '- 충남 보령 외연도, 태안 옹도 부근, 서산 부석면 연안',
 '- 전북 군산 어청도 부근',
 '- 전남 진도 원포리, 완도 월양리 연안,',
 '- 경남 고성 송천리, 남해 진목리 연안, 통영 욕지도 부근 수월리 연안, 창원 심리, 요장리, 덕동동 연안',
 '- 부산 가덕도 부근',
 '- 울산 주전동 연안',
 '- 경북 포항 호미곶면, 남호리, 지경리 연안, 울진 덕천리 연안, 경주 감포 연안',
 '- 강원 속초 용촌리 연안, 강릉 주문진 연안']

In [11]:
wheret1 = [i.replace(' 연안', '') for i in wheret]
wheret1 = [i.replace(' 부근', '') for i in wheret1]
wheret1 = [i.replace(' 등대', '') for i in wheret1]
wheret1 = [i.replace(',,', ',') for i in wheret1]
wheret1

['○동․서․남동해 출현',
 '•고밀도 출현 해역',
 '- 충남 당진 난지도, 태안 황도리, 서천 마량리',
 '- 전북 군산 십이동파도, 새만금 내측·외측, 비응도, 신시도, 무녀도, 부안 치도리, 진리',
 '- 전남 고흥 도천리, 안남리, 거금도, 보성 비봉리, 회천면, 영암 삼호읍',
 '- 부산 기장 동수',
 '- 경남 고성 수월리, 덕명리, 당항포, 창원 잠도, 수도, 행암동 난포리, 거제 지심도, 구조라해변, 사등면',
 '- 경북 울진 오산항',
 '•저밀도 출현 해역',
 '- 충남 보령 외연도, 태안 옹도, 서산 부석면',
 '- 전북 군산 어청도',
 '- 전남 진도 원포리, 완도 월양리,',
 '- 경남 고성 송천리, 남해 진목리, 통영 욕지도 수월리, 창원 심리, 요장리, 덕동동',
 '- 부산 가덕도',
 '- 울산 주전동',
 '- 경북 포항 호미곶면, 남호리, 지경리, 울진 덕천리, 경주 감포',
 '- 강원 속초 용촌리, 강릉 주문진']

In [16]:
idx = []
for i in range(len(wheret1)):
    if '-' in wheret1[i]:
        idx.append(i)
        
print(idx)

wheret2 = [i.split(', ') for i in wheret1]
wheret2

[2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16]


[['○동․서․남동해 출현'],
 ['•고밀도 출현 해역'],
 ['- 충남 당진 난지도', '태안 황도리', '서천 마량리'],
 ['- 전북 군산 십이동파도', '새만금 내측·외측', '비응도', '신시도', '무녀도', '부안 치도리', '진리'],
 ['- 전남 고흥 도천리', '안남리', '거금도', '보성 비봉리', '회천면', '영암 삼호읍'],
 ['- 부산 기장 동수'],
 ['- 경남 고성 수월리',
  '덕명리',
  '당항포',
  '창원 잠도',
  '수도',
  '행암동 난포리',
  '거제 지심도',
  '구조라해변',
  '사등면'],
 ['- 경북 울진 오산항'],
 ['•저밀도 출현 해역'],
 ['- 충남 보령 외연도', '태안 옹도', '서산 부석면'],
 ['- 전북 군산 어청도'],
 ['- 전남 진도 원포리', '완도 월양리,'],
 ['- 경남 고성 송천리', '남해 진목리', '통영 욕지도 수월리', '창원 심리', '요장리', '덕동동'],
 ['- 부산 가덕도'],
 ['- 울산 주전동'],
 ['- 경북 포항 호미곶면', '남호리', '지경리', '울진 덕천리', '경주 감포'],
 ['- 강원 속초 용촌리', '강릉 주문진']]

In [17]:
for i in idx:
    
    if len(wheret2[i][0].split(' ')) >= 4:
        middle = wheret2[i][0].split(' ')[2] + ' '
    else:
        middle = ''
    
    
    for j in range(1, len(wheret2[i])):
        
        if len(wheret2[i][j].split(' ')) >= 2:
            wheret2[i][j] = '- ' + wheret2[i][0].split(' ')[1] +' '+ wheret2[i][j]
            middle = wheret2[i][j].split(' ')[2] +' '

        if len(wheret2[i][j].split(' ')) == 1:
            wheret2[i][j] = '- ' + wheret2[i][0].split(' ')[1] +' '+ middle+ wheret2[i][j]
            
wheret2

[['○동․서․남동해 출현'],
 ['•고밀도 출현 해역'],
 ['- 충남 당진 난지도', '- 충남 태안 황도리', '- 충남 서천 마량리'],
 ['- 전북 군산 십이동파도',
  '- 전북 새만금 내측·외측',
  '- 전북 새만금 비응도',
  '- 전북 새만금 신시도',
  '- 전북 새만금 무녀도',
  '- 전북 부안 치도리',
  '- 전북 부안 진리'],
 ['- 전남 고흥 도천리',
  '- 전남 고흥 안남리',
  '- 전남 고흥 거금도',
  '- 전남 보성 비봉리',
  '- 전남 보성 회천면',
  '- 전남 영암 삼호읍'],
 ['- 부산 기장 동수'],
 ['- 경남 고성 수월리',
  '- 경남 고성 덕명리',
  '- 경남 고성 당항포',
  '- 경남 창원 잠도',
  '- 경남 창원 수도',
  '- 경남 행암동 난포리',
  '- 경남 거제 지심도',
  '- 경남 거제 구조라해변',
  '- 경남 거제 사등면'],
 ['- 경북 울진 오산항'],
 ['•저밀도 출현 해역'],
 ['- 충남 보령 외연도', '- 충남 태안 옹도', '- 충남 서산 부석면'],
 ['- 전북 군산 어청도'],
 ['- 전남 진도 원포리', '- 전남 완도 월양리,'],
 ['- 경남 고성 송천리',
  '- 경남 남해 진목리',
  '- 경남 통영 욕지도 수월리',
  '- 경남 창원 심리',
  '- 경남 창원 요장리',
  '- 경남 창원 덕동동'],
 ['- 부산 가덕도'],
 ['- 울산 주전동'],
 ['- 경북 포항 호미곶면', '- 경북 포항 남호리', '- 경북 포항 지경리', '- 경북 울진 덕천리', '- 경북 경주 감포'],
 ['- 강원 속초 용촌리', '- 강원 강릉 주문진']]

In [ ]:
idx = []
for i in range(len(wheret1)):
    if '-' in wheret1[i]:
        idx.append(i)
        

wheret3 = [i.split(', ') for i in wheret1]
wheret3

for i in idx:
    
    for j in range(1, len(wheret3[i])):
         wheret3[i][j] = '- ' + wheret3[i][0].split(' ')[1] +' '+ wheret3[i][j]
            
wheret3

In [18]:
def flatten(lst):
    result = []
    for item in lst:
        result.extend(item)
    return result

wheret4 = flatten(wheret2)
wheret4

['○동․서․남동해 출현',
 '•고밀도 출현 해역',
 '- 충남 당진 난지도',
 '- 충남 태안 황도리',
 '- 충남 서천 마량리',
 '- 전북 군산 십이동파도',
 '- 전북 새만금 내측·외측',
 '- 전북 새만금 비응도',
 '- 전북 새만금 신시도',
 '- 전북 새만금 무녀도',
 '- 전북 부안 치도리',
 '- 전북 부안 진리',
 '- 전남 고흥 도천리',
 '- 전남 고흥 안남리',
 '- 전남 고흥 거금도',
 '- 전남 보성 비봉리',
 '- 전남 보성 회천면',
 '- 전남 영암 삼호읍',
 '- 부산 기장 동수',
 '- 경남 고성 수월리',
 '- 경남 고성 덕명리',
 '- 경남 고성 당항포',
 '- 경남 창원 잠도',
 '- 경남 창원 수도',
 '- 경남 행암동 난포리',
 '- 경남 거제 지심도',
 '- 경남 거제 구조라해변',
 '- 경남 거제 사등면',
 '- 경북 울진 오산항',
 '•저밀도 출현 해역',
 '- 충남 보령 외연도',
 '- 충남 태안 옹도',
 '- 충남 서산 부석면',
 '- 전북 군산 어청도',
 '- 전남 진도 원포리',
 '- 전남 완도 월양리,',
 '- 경남 고성 송천리',
 '- 경남 남해 진목리',
 '- 경남 통영 욕지도 수월리',
 '- 경남 창원 심리',
 '- 경남 창원 요장리',
 '- 경남 창원 덕동동',
 '- 부산 가덕도',
 '- 울산 주전동',
 '- 경북 포항 호미곶면',
 '- 경북 포항 남호리',
 '- 경북 포항 지경리',
 '- 경북 울진 덕천리',
 '- 경북 경주 감포',
 '- 강원 속초 용촌리',
 '- 강원 강릉 주문진']

In [ ]:
wheret4

In [22]:
def flatten(lst):
    result = []
    for item in lst:
        result.extend(item)
    return result

wheret4 = flatten(wheret2)
wheret5 = wheret4


for i in range(len(wheret5)):
    if '-' in wheret5[i]:
        for j in range(i+1, len(wheret5)):
            if '-' in wheret5[j]:
                if wheret5[i] == wheret5[j]:
                    wheret5[j] = '0'
            else:
                break
                
wheret5
                
                

['○동․서․남동해 출현',
 '•고밀도 출현 해역',
 '- 충남 당진 난지도',
 '- 충남 태안 황도리',
 '- 충남 서천 마량리',
 '- 전북 군산 십이동파도',
 '- 전북 새만금 내측·외측',
 '- 전북 새만금 비응도',
 '- 전북 새만금 신시도',
 '- 전북 새만금 무녀도',
 '- 전북 부안 치도리',
 '- 전북 부안 진리',
 '- 전남 고흥 도천리',
 '- 전남 고흥 안남리',
 '- 전남 고흥 거금도',
 '- 전남 보성 비봉리',
 '- 전남 보성 회천면',
 '- 전남 영암 삼호읍',
 '- 부산 기장 동수',
 '- 경남 고성 수월리',
 '- 경남 고성 덕명리',
 '- 경남 고성 당항포',
 '- 경남 창원 잠도',
 '- 경남 창원 수도',
 '- 경남 행암동 난포리',
 '- 경남 거제 지심도',
 '- 경남 거제 구조라해변',
 '- 경남 거제 사등면',
 '- 경북 울진 오산항',
 '•저밀도 출현 해역',
 '- 충남 보령 외연도',
 '- 충남 태안 옹도',
 '- 충남 서산 부석면',
 '- 전북 군산 어청도',
 '- 전남 진도 원포리',
 '- 전남 완도 월양리,',
 '- 경남 고성 송천리',
 '- 경남 남해 진목리',
 '- 경남 통영 욕지도 수월리',
 '- 경남 창원 심리',
 '- 경남 창원 요장리',
 '- 경남 창원 덕동동',
 '- 부산 가덕도',
 '- 울산 주전동',
 '- 경북 포항 호미곶면',
 '- 경북 포항 남호리',
 '- 경북 포항 지경리',
 '- 경북 울진 덕천리',
 '- 경북 경주 감포',
 '- 강원 속초 용촌리',
 '- 강원 강릉 주문진']

In [24]:
wheret6 = [item for item in wheret5 if item != '0']
wheret6

['○동․서․남동해 출현',
 '•고밀도 출현 해역',
 '- 충남 당진 난지도',
 '- 충남 태안 황도리',
 '- 충남 서천 마량리',
 '- 전북 군산 십이동파도',
 '- 전북 새만금 내측·외측',
 '- 전북 새만금 비응도',
 '- 전북 새만금 신시도',
 '- 전북 새만금 무녀도',
 '- 전북 부안 치도리',
 '- 전북 부안 진리',
 '- 전남 고흥 도천리',
 '- 전남 고흥 안남리',
 '- 전남 고흥 거금도',
 '- 전남 보성 비봉리',
 '- 전남 보성 회천면',
 '- 전남 영암 삼호읍',
 '- 부산 기장 동수',
 '- 경남 고성 수월리',
 '- 경남 고성 덕명리',
 '- 경남 고성 당항포',
 '- 경남 창원 잠도',
 '- 경남 창원 수도',
 '- 경남 행암동 난포리',
 '- 경남 거제 지심도',
 '- 경남 거제 구조라해변',
 '- 경남 거제 사등면',
 '- 경북 울진 오산항',
 '•저밀도 출현 해역',
 '- 충남 보령 외연도',
 '- 충남 태안 옹도',
 '- 충남 서산 부석면',
 '- 전북 군산 어청도',
 '- 전남 진도 원포리',
 '- 전남 완도 월양리,',
 '- 경남 고성 송천리',
 '- 경남 남해 진목리',
 '- 경남 통영 욕지도 수월리',
 '- 경남 창원 심리',
 '- 경남 창원 요장리',
 '- 경남 창원 덕동동',
 '- 부산 가덕도',
 '- 울산 주전동',
 '- 경북 포항 호미곶면',
 '- 경북 포항 남호리',
 '- 경북 포항 지경리',
 '- 경북 울진 덕천리',
 '- 경북 경주 감포',
 '- 강원 속초 용촌리',
 '- 강원 강릉 주문진']

In [ ]:
wheret4

In [ ]:
wheret4_1

In [ ]:
import re

br = re.compile('(?<=\<br>|?<=\<br/>)(.*?)\r')

a = table[1].find_all('tr')[3].find_all('td')[1]
br.findall(str(a))

In [ ]:
str(a)

In [ ]:
table[1].find_all('tr')[len(table[1].find_all('tr'))-1]

In [ ]:
table[1].find_all('tr')[2]